# BE 1 traitement et protection de l'information

## 1. Huffman Algorithm

In [91]:
clear all; close all; clc;

1. We first write the function `DMS(A, P, m, n)` that computes a discrete source matrix of size $m \times n$ from an alphabet $A$ and a probability vector $P$.

In [92]:
%%file DMS.m
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% function DMS =  DMS(A, P, m, n)
% generate discrete source matrix from alphabet A and probabilities P
% m: number of rows, n: number of columns
% DMS: discrete source matrix
% returns DMS
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
function DMS =  DMS(A, P, m, n)
    DMS = randsrc(m, n, [A; P]);
end

Created file '/Users/thomasprevost/github/ProtectInfo/BE1/DMS.m'.


We then try it with the following alphabet and probability vector, and compare the result with the one obtained with the `repmat` function.

In [93]:
A = [0, 1];
P = [0.5, 0.5];
m = 5;
n = 6;
X = DMS(A, P, m, n);

In [94]:
Y = repmat(A, m, n);

In [95]:
X
Y


X =

     0     0     0     1     0     1
     0     1     0     0     1     0
     1     0     0     0     1     1
     0     1     0     0     0     1
     0     0     0     1     0     0


Y =

     0     1     0     1     0     1     0     1     0     1     0     1
     0     1     0     1     0     1     0     1     0     1     0     1
     0     1     0     1     0     1     0     1     0     1     0     1
     0     1     0     1     0     1     0     1     0     1     0     1
     0     1     0     1     0     1     0     1     0     1     0     1



2. That being done, we write the function `entropy(P)` that computes the entropy of a discrete random variable from its probability vector, and test it with a probability vector.

In [96]:
%%file entropy.m
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% function H = entropy(P)
% compute entropy of discrete random variable
% P: probability vector
% return H
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
function H = entropy(P)
    H = -sum(P .* log2(P));
end

Created file '/Users/thomasprevost/github/ProtectInfo/BE1/entropy.m'.


In [97]:
P = [.2, .5, .1, .2];
entropy(P)


ans =

    1.7610



3. We write the function `moybits(N, P)` that computes the average number of bits per symbol from a probability vector and a vector of codeword lengths.

In [98]:
%%file moybits.m
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% function moybits = moybits(N, P)
% compute average number of bits per symbol
% P: probability vector
% N: length of codewords
% moybits: average number of bits per symbol
% returns moybits
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
function moybits = moybits(N, P)
    moybits = sum(P .* N);
end


Created file '/Users/thomasprevost/github/ProtectInfo/BE1/moybits.m'.


4. Now, we try Matlab functions `huffmanenco` and `huffmandeco` to encode and decode a discrete source matrix.

In [99]:
% test alphabet and probability vector
A = [1:6];
P = [.05, .125, .25, .12, .3, .155];

% discrete source
X = DMS(A, P, 3, 3);

% create dictionary
dict = huffmandict(A, P);

In [100]:
% reshape X to a vector
X = reshape(X, 1, numel(X));
X

% encode
E = huffmanenco(X, dict)

% decode
D = huffmandeco(E,dict)


X =

     5     5     2     3     6     3     1     4     1


E =

  Columns 1 through 13

     0     0     0     0     0     1     1     1     0     0     1     0     1

  Columns 14 through 23

     0     1     1     1     1     1     0     1     1     1


D =

     5     5     2     3     6     3     1     4     1



We then try the same thing with different alphabets and probability vectors.

In [101]:
A = [1:6];
P = [.05, .125, .25, .12, .3, .155];

A2 = [5:9];
P2 = [.05, .125, .25, .275, .3];

X = DMS(A, P, 3, 4);
X2 = DMS(A2, P2, 3, 4);

dict = huffmandict(A, P);
dict2 = huffmandict(A2, P2);

X = reshape(X, 1, numel(X))
E = huffmanenco(X, dict)
D = huffmandeco(E,dict)

X2 = reshape(X2, 1, numel(X2))
E2 = huffmanenco(X2, dict2)
D2 = huffmandeco(E2,dict2)


X =

     5     4     6     5     2     5     2     6     1     5     5     5


E =

  Columns 1 through 13

     0     0     1     1     0     0     1     0     0     0     0     1     1

  Columns 14 through 26

     0     0     0     1     1     0     1     0     1     1     1     0     0

  Columns 27 through 30

     0     0     0     0


D =

     5     4     6     5     2     5     2     6     1     5     5     5


X2 =

     7     9     8     8     6     9     8     9     9     7     6     6


E2 =

  Columns 1 through 13

     1     0     0     0     0     1     0     1     1     1     0     0     0

  Columns 14 through 26

     0     1     0     0     0     0     1     0     1     1     0     1     1

  Column 27

     0


D2 =

     7     9     8     8     6     9     8     9     9     7     6     6



We now check the Kraft inequality for the two examples.

In [102]:
% lengths of codewords
N = cellfun(@length, dict(:,2));
N2 = cellfun(@length, dict2(:,2));

%lengths of alphabets
L = length(A);
L2 = length(A2);

% kraft inequality
kraft = sum(L.^(-N))
kraft2 = sum(L2.^(-N2))


kraft =

    0.0741


kraft2 =

    0.1360



5. We use the functions `arithenco` and `arithdeco` to encode and decode a discrete source matrix on several alphabets and probability vectors.

In [103]:
A = [1:4];
P = [.1 .5 .3 .1];
X = DMS(A, P, 1, 500);
E = arithenco(X, [sum(X==1) sum(X==2) sum(X==3) sum(X==4)]);
D = arithdeco(E, [sum(X==1) sum(X==2) sum(X==3) sum(X==4)], 500);
"sum D - X = " + sum(D - X)

A2 = [1:6];
P2 = [.1 .2 .3 .1 .2 .1];
X2 = DMS(A2, P2, 1, 500);
E2 = arithenco(X2, [sum(X2==1) sum(X2==2) sum(X2==3) sum(X2==4) sum(X2==5) sum(X2==6)]);
D2 = arithdeco(E2, [sum(X2==1) sum(X2==2) sum(X2==3) sum(X2==4) sum(X2==5) sum(X2==6)], 500);
"sum D2 - X2 = " + sum(D2 - X2)


ans = 

    "sum D - X = 0"


ans = 

    "sum D2 - X2 = 0"



=> Encoding and decoding work fine, messages are the same before and after encoding and decoding.

In [104]:
% Kraft inequality
L = length(A);
L2 = length(A2);
N = ceil(log2(L));
N2 = ceil(log2(L2));
kraft = sum(L.^(-N))
kraft2 = sum(L2.^(-N2))


kraft =

    0.0625


kraft2 =

    0.0046

